<a href="https://colab.research.google.com/github/nanom/llm_adaptation_workshop/blob/main/Taller_Modelos_de_lenguaje_a_tu_medida_13_de_septiembre_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller: Modelos de lenguaje a tu medida 🤖💬

¿Quiénes somos?
------------------

* **Hernán Maina** - Becario doctoral de CONICET ( [🤗](https://huggingface.co/nanom) | [🌐](https://nanom.github.io/) [📫](mailto:hernan.maina@mi.unc.edu.ar))
* **Guido Ivetta** - Becario doctoral de Vía Libre y Profesor Ayudante de FAMAF ([🤗](https://huggingface.co/guidoivetta) | [📫](mailto:guidoivetta@mi.unc.edu.ar ))
* **Laura Alonso Alemany** - Profesora Asociada de FAMAF ([🐦](@morlaicassiopea) | [🌐](https://www.cs.famaf.unc.edu.ar/~laura/) | [📫](mailto:lauraalonsoalemany@unc.edu.ar))
* **Luciana Benotti** - Profesora Asociada de FAMAF ([🐦](@LucianaBenotti) | [🌐](https://benotti.github.io/) | [📫](mailto:luciana.benotti@unc.edu.ar))

<div style="text-align:center">
 <h4 style="font-size:1.5em;margin:5px"></h4>
    <h5 style="font-style:normal;font-size:1em;margin:5px"></h5>
    <div style="display:inline-block;margin-right:0px;">
        <img src="https://vialibre.org.ar/wp-content/uploads/2020/10/banner-2.jpg" style="height:10em;width:auto;"/>
    </div>
    <h6 style="font-style:normal;font-size:0.9em;margin:5px;">
        <a href="https://twitter.com/" style="color:royalblue;" target="_blank"> @fvialibre</a> -
        <a href="https://www.vialibre.org.ar/" style="color:royalblue;" target="_blank">https://www.vialibre.org.ar/</a>
    </h6>
</div>

## Esquema del taller

1. [Modelos de lenguaje](#Modelos-de-lenguaje)
2. [Librerias](#Librerias)
3. [Probemos modelos](#Probemos-modelos)
4. [Personalicemos un modelo](#Personalicemos-un-modelo)

*Requerimientos deseables para entender este taller:*
* Conocimiento básico de Python
* Conocimiento básico en entrenamiento de modelos

*Librerías principales utilizadas*:
* [🤗 HuggingFace](https://huggingface.co/)

*Agradecimientos*:
* **Cristian Cardellino**, por la inspiración en la estructura de partes del notebook. Ver [acá](https://colab.research.google.com/#fileId=https%3A//huggingface.co/crscardellino/xi-ciai-cba-martin-fierro/blob/main/xi-ciai-cba.ipynb) para más detalles.
* **Beatriz Busaniche**, **Nair Carolina** y **Alexia Halvorsen** de la Fundación Vía Libre por su continuo apoyo.
* **Matías Bordone** y **Librebase Córdoba**, por la organización de la jornada.
* **Nicolás Wolovick** y a todo el equipo de **CCAD-UNC** por disponibilizar recursos para la creación y desarrollo de este taller.
* **FAMAF**, por la apertura y predisposición.

# **IMPORTANTE**

* Todas las celdas indicadas con el símbolo ( ❗ ) son obligatorias y necesarias para el correcto funcionamiento de este taller.

* Aquellas señaladas con ( 🔎 ) son de profundización de contenido, optativas de leer y computar.

* Si una celda contiene el símbolo ( ⏳ ), significa que tomará más tiempo en computarse que el promedio. Tener en cuenta para aprovechar al máximo el tiempo disponible del taller.

# 0 - Inicialización de la notebook

In [ ]:
# @markdown ❗
# Instalación de librerías
!pip install -U transformers[torch] --quiet
!pip install -U tabulate --quiet
!pip install -U datasets --quiet

In [ ]:
# @markdown ❗
# Descarga de datasets
import os

ROOT_PATH = "llm_adaptation_workshop"
DATASETS_PATH = os.path.join(ROOT_PATH, "datasets")

%rm -r "$ROOT_PATH"
!git clone https://github.com/nanom/llm_adaptation_workshop.git "$ROOT_PATH"

In [ ]:
# @markdown ❗
# Comprobación de recursos (GPUs)
!nvidia-smi

# 1 - Modelos de lenguaje

## ¿Qué son los "modelos de lenguaje"?

- Son modelos estadísticos predictivos, basados en aprendizaje automático sobre textos.
- Su función principal es **analizar** y/o **generar** texto de manera automática.
- Se entrenan encontrando patrones en grandes cantidades de texto libre.
- Dado un contexto (e.g. una secuencia de palabras), aplican los patrones inferidos para predecir la palabra siguiente, generando texto plausible y coherente.
- Si bien los modelos de lenguaje existen desde hace varias décadas en diferentes formas (modelos markovianos, conditional random fields, redes neuronales recurrentes), actualmente cuando alguien habla de un "Modelo de Lenguaje", usualmente se refiere a un modelo neuronal de tipo **Transformer**.

### ¿Qué es un **Transformer**?

- Es un tipo de arquitectura de redes neuronales que se introdujo en el paper ["Attention is All You Need"](https://arxiv.org/abs/1706.03762).

- El tipo de aprendizaje que hacen estas redes está basado en **mecanismos de atención**, que ayudan al modelo a enfocarse en las partes más determinantes de la información, de forma que el entrenamiento sea más rápido y menos costoso que para otras arquitecturas, como las redes neuronales recurrentes.

<!-- La idea fundamental detras de un modelo Transformer, es procesar el lenguaje natural de una manera muy eficiente y efectiva, haciendolo ideal para ser aplicado a una gran cantidad de tareas de PLN.-->

- Existen diferentes variantes de transformer, de acuerdo a cómo y qué parte de sus componentes utilizan:
    - Los modelos de traducción secuencia a secuencia o `Seq2Seq` (e.g. [T5](https://arxiv.org/abs/1910.10683)), tienen un **codificador y decodificador** y son empleados para tareas de transformación como traducción, simplificación, cambio de estilo o resumen.
    - Los modelos que sólo usan el **codificador** (e.g. [BERT](https://arxiv.org/abs/1810.04805)) se usan para obtener representaciones vectoriales del texto (*embeddings*) que resultan muy útiles para  determinar relaciones de semejanza entre diferentes textos.
    - Los modelos basados en el **decodificador** (e.g. [GPT](https://arxiv.org/abs/2005.14165)) se usan para generar texto automáticamente, como respuestas a preguntas, ensayos o cuentos, entre otros.

<center><img src='https://heidloff.net/assets/img/2023/02/transformers.png' width=60%></center>

**Nota:** *Para una explicación más sencilla, pero más detallada, sugiero los posts de la serie "The Illustrated..." de [Jay Alammar](http://jalammar.github.io/):*
- [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [The Illustrated BERT](http://jalammar.github.io/illustrated-bert/)
- [The Illustrated GPT-2](http://jalammar.github.io/illustrated-gpt2/) / [How GPT-3 Works](http://jalammar.github.io/how-gpt3-works-visualizations-animations/)

## Probamos un BERT mediante la tarea *fill-mask*

En esta sección vamos a practicar con [BERT](https://en.wikipedia.org/wiki/BERT_(language_model)), un modelo de lenguaje con una arquitectura compuesta de **codificadores** (encoders).


Para familiarizarnos con BERT, vamos a jugar con la tarea "*Fill-Mask*", cuyo objetivo es completar una oración con algunas palabras, en los lugares marcados con `[MASK]`.

El modelo de lenguaje predecirá las palabras más adecuadas para encajar en esos espacios en blanco, es decir, las más probables dado el contexto, según los patrones estadísticos inferidos de los ejemplos de aprendizaje. Esto nos sirve para evaluar qué tan bien está modelando los textos.

In [11]:
# @markdown 🔎
# Usamos pipeline como función auxiliar de alto nivel
from transformers import pipeline

fill_mask = pipeline(
    task="fill-mask",
    model="dccuchile/bert-base-spanish-wwm-uncased",
    top_k=5
)

In [ ]:
# @markdown 🔎
ejemplo = 'Córdoba es una [MASK] de Argentina.'
fill_mask(ejemplo)

## Probamos un GPT para la tarea de generación de texto

En esta sección vamos a practicar con [GPT](https://en.wikipedia.org/wiki/Generative_pre-trained_transformer), un modelo de lenguaje generativo con una arquitectura basada en transformers.

Vamos a jugar con la función de generación del modelo, que, dada una secuencia de palabras, predice la continuación más probable para la misma, según los patrones inferidos en los ejemplos de aprendizaje.

In [9]:
# @markdown 🔎
# Usamos pipeline como función auxiliar de alto nivel
from transformers import pipeline

text_gen = pipeline(
    task="text-generation",
    model="gpt2",
    pad_token_id=50256
)

In [ ]:
# @markdown 🔎
ejemplo="My name is Lewis and I like to"
text_gen(ejemplo)

# 2 - Librerías para modelos de lenguaje


##  ¿Qué es Hugging Face 🤗?

- Una [comunidad colaborativa](https://huggingface.co/) especialmente enfocada en modelos de lenguaje y otros recursos de Inteligencia Artificial (IA).
- Ofrece repositorios para disponibilizar [modelos](https://huggingface.co/models), [datasets](https://huggingface.co/datasets) y [demos](https://huggingface.co/spaces).
- Además, ofrece varias librerías orientadas a la IA, particularmente al Aprendizaje Profundo (*Deep Learning*), entre las que destacan:
    - [`transformers`](https://huggingface.co/docs/transformers): La que veremos en esta charla, para todo lo relacionado a Procesamiento del Lenguaje Natural (PLN) con grandes modelos de lenguaje (*Large Language Models*, LLMs).
    - [`datasets`](https://huggingface.co/docs/datasets): Una librería con funcionalidades para el tratamiento de los conjuntos de datos a utilizar para entrenar o ajustar los LLMs.
    - [`tokenizers`](https://huggingface.co/docs/tokenizers): Una librería para el proceso de "tokenización", i.e. la división de texto de manera discreta en palabras o subpalabras.
- Hugging Face no sólo ofrece soluciones para PLN, sino también para imágenes, con librerías como [`diffusers`](https://huggingface.co/docs/diffusers), para la generación de imágenes:
    - Lectura recomendada: [The Illustrated Stable Diffusion](http://jalammar.github.io/illustrated-stable-diffusion/)

## ¿Cómo empezar con Hugging Face 🤗?

- Primero se [crea una cuenta en la página](https://huggingface.co/join).
- Luego podemos [crear modelos](https://huggingface.co/new) a través del menú que se despliega de nuestro avatar.
- Para poder subir el modelo personalizado que entrenaremos en esta notebook en tu cuenta de Hugging Face, necesitarás generar un token de acceso mediante los siguentes pasos:
    1. **Accedé** a la [sección de tokens de acceso](https://huggingface.co/settings/tokens) de tu perfil
    <center><img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/User-Access-Token.png" width=70%></center>
    2. Creá un **nuevo token** de acceso con permiso de escritura.
    <center><img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/new-token.png" width=60%></center>
    3. **Listo!**. Ya tenés tu token preparado para poder subir tu modelo a la plataforma de Hugging Face 🤗.

# 3 - ¿Cómo se entrenan los modelos de lenguaje?

En esta notebook **NO** vamos a entrenar completamente un LLM, porque los LLMs requieren de muchos datos y mucho cómputo para ser entrenados:
- Para **BERT**:
    - El costo total estimado de entrenamiento del modelo fue de U\$D 6912 (para su version *large*) y de U\$D 500 (para su versión *base*).
    - Mientras que la versión pequeña (*base*) cuenta con 109M de parámetros, su versión original (*large*) oscila en 334M, más del triple.
    - Su entrenamiento fue realizado sobre 3.3B de tokens (aproximadamente 20 GB de texto no comprimido).

- Para **GPT-3**:
    - Se estimó un costo de entrenamiento cercano a los U\$D 4.6 Millones.
    - Disponible en ocho tamaños, que van desde los 125M a los 175B parámetros.
    - Requirió de varias semanas de entrenamiento.
    - El corpus reportado en el cual fue entrenado es aproximadamente 500B de palabras.
    - Varios GPUS para entrenarlo y hardware especializado.

Sin embargo, vamos a **entrenar parcialmente** un modelo de lenguaje, en concreto, vamos a adaptarlo a un dominio particular.



## Diferencia entre pre-training y fine-tuning

La distinción clave entre pre-training y fine-tuning se encuentra en las etapas del proceso de entrenamiento:

* El **pre-training**, es la primera fase de entrenamiento al que se somete todo modelo LLM, y requiere de una enorme cantidad de textos genéricos, como se ha descrito en el apartado anterior.

* Una vez finalizado este proceso inicial de entrenamiento, se obtiene un modelo genérico capaz de realizar predicciones genéricas sobre patrones textuales, pero sin especialización en ninguna tareas específicas.

* Si deseamos adaptar este modelo genérico para que desempeñe una tarea específica con un alto rendimiento (e.g.: *Text Clasification*, *Sentiment analysis*, *Question Answering*, *Information Extraction*, etc.), se lleva a cabo una segunda etapa de entrenamiento conocida como **fine-tuning**.

<center>
<!-- <img src="https://serokell.io/files/7i/7iyrq1z5.Inside_ChatGPT_pic1.png" width=80%>-->
<img src="https://miro.medium.com/v2/resize:fit:951/0*R31A71UjHM8R8Pps.png" width=80%>
</center>
<right> fuente: https://medium.com/mantisnlp/supervised-fine-tuning-customizing-llms-a2c1edbf22c3
</right>

Este entrenamiento específico requiere de menos ejemplos de entrenamiento y menos cálculo, ya que sólo modifica ligeramente el modelo ya entrenado. En este proceso, existen varías formas en como se "modifica" el modelo pre-entrenado durante esta fase: se pueden modificar las últimas capas, añadir capas a la red pre-entrenada, y otras variantes. En esta notebook vamos a trabajar con una aproximación sencilla, pero existen múltiples librerías en Huggingface que implementan funcionalidades para llevar adelante difrentes tipos de *fine-tuning*, como por ejemplo la popular [LoRA](https://www.databricks.com/blog/efficient-fine-tuning-lora-guide-llms).

<!--<center>
<img src="https://media.licdn.com/dms/image/D5612AQEe2e_DJ51f2g/article-inline_image-shrink_1000_1488/0/1687324002269?e=1699488000&v=beta&t=vWDbM2RYNKa9tv5--w6Iub7YRRGkjld-s7U6VDd_7_Q" width=80%>

</center>
<right> fuente: https://www.linkedin.com/pulse/beginners-guide-fine-tuning-large-language-models-vaidheeswaran
</right>-->

## *Domain Adaptation*

Mientras que *fine-tuning* se centra en ajustar un modelo de lenguaje *pre-entrenado* para realizar tareas específicas, la adaptación de dominio se enfoca en hacer que el modelo sea más efectivo en un dominio de datos particular.

En caso de que tus datos de entrenamiento pertenezcan a un determinado tema específico (e.g.: leyes, medicina, informática, etc.) o un estilo específico (e.g.: académico, infantil, publicitario, etc.), y difieran substancialmente del corpus estándar en el cual fue entrenado inicialmente el LLM, podemos pensar diferentes opciones para incorparar este nuevo conocimiento:


1. **Entrenamiento desde cero:** Este enfoque implica entrenar desde cero (*from scratch*) un nuevo modelo de lenguaje con textos del dominio de interés. Sin embargo, no es recomendable para el público en general, dada la gran cantidad de recursos computacionales, horas de procesamiento y textos de entrenamiento necesarios para lograr un rendimiento comparable a los modelos estado del arte (modelos *state-of-the-art, SOTA*).
2.  **Uso de modelos disponibles:** Se aprovechan modelos disponibles públicamente, específicos para el dominio de interés (e.g: [LegalBERT](https://arxiv.org/abs/2010.02559), [FinBERT](https://arxiv.org/abs/1908.10063) y [BioBERT](https://arxiv.org/abs/1901.08746); todos disponibles en Hugging Face 🤗 de manera gratuita.)
3. **Adaptación de modelos pre-entrenados**: Esta aproximación, tambien conocida en la literatura como *'further pre-training'*, *'inter-training'*, *'continued pre-training'* o *'domain-adaptation'*, implica tomar un modelo de lenguaje pre-entrenado para un dominio cualquiera y, aprovechando todo el conocimiento y las representaciones ya aprendidas, **continuar su entrenamiento sobre el conjunto de datos especializados o personalizados**. Esta técnica permite alcanzar muy buenos resultados, utilizando menos recursos computacionales, horas y datos de entrenamiento.

<center>
<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*4eNaH6y0dSsaxtTQW0tgHQ.png" width=80%>
</center>
<right> fuente: https://medium.com/@shankar.arunp/training-bert-from-scratch-on-your-custom-domain-data-a-step-by-step-guide-with-amazon-25fcbee4316a
</right>

# 4 - Vamos a personalizar un modelo!

## 4.1 - ¿Cómo personalizar un modelo BERT?


Tanto para el entrenamiento inicial (*pre-training*) como para la adaptación de dominio mediante un entrenamiento intermedio (*inter-training*), el modelo debe ser re-entrenado utilizando tareas auto-supervisadas, ya sea la tarea de **Masked Language Modeling (MLM)**, o una combinación entre **(MLM)** y **Next Sentence Prediction (NSP)**.

### Masked Language Modeling (MLM)
> Consiste en enmascarar (ocultar) palabras aleatorias en cada oración o frase de entrada, y entrenar el modelo para que aprenda a predecirlas basándose en el contexto circundante (palabras vecinas). Durante el entrenamiento, alrededor del **15%** de las palabras se seleccionan al azar y se enmascaran, reemplazándolas con el token especial **[MASK]**. De esta forma, el modelo infiere patrones de relaciones entre palabras.
<center>
<img src="http://jalammar.github.io/images/BERT-language-modeling-masked-lm.png" width=65%>
</center>

### Next Sentence Prediction (NSP)
> Aunque no obligatorio, es otro componente importante en el entrenamiento de BERT. A partir de un texto considerado como una secuencia de oraciones, se generan ejemplos de entrenamiento consistentes en pares de oraciones aleatorios. El modelo tiene que aprender a identificar si una oración sigue a otra en el texto original. De esta forma, el modelo infiere patrones de relaciones entre oraciones.

<center>
<img src="http://jalammar.github.io/images/bert-next-sentence-prediction.png" width=65%>
</center>

En esta notebook **realizaremos el ajuste de dominio mediante la tarea de MLM** utilizando la clase [**BertForMaskedLM**](https://huggingface.co/docs/transformers/v4.32.1/en/model_doc/bert#transformers.BertForMaskedLM) (modelo BERT con un bloque superior extra que posibilita el `modelado del lenguaje enmascarado`), ofrecida por la biblioteca [**transformer**](https://github.com/huggingface/transformers) de Hugging Face 🤗.

* Se utilizarán diferentes datasets especializados (tanto en español como en inglés).
* Utilizaremos como modelo pre-entrenado  [bert-base-uncased](https://huggingface.co/bert-base-uncased) para realizar la adaptación en los datasets en inglés, y [dccuchile/bert-base-spanish-wwm-uncased](https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased) para los correspondientes en español.


### Carga de datasets

* Utilizaremos la librería `datasets` de HuggingFace para cargar cada uno de los corpus seleccionados.
* Para la personalización de BERT, tenemos un total de 4 conjuntos de datos, 3 de ellos en español y 1 en Inglés:
    * Peppa Pig (es):
      * ≈ 3k muestras.
      * Extraído de subtítulos de 77 episodios.
    * Martin Fierro (es):
      * ≈ 2k muestras.
      * Extraído del libro completo de José Hernández
    * Preguntas de no videntes (en):
      * ≈ 33k muestras.
      * Generado a partir de preguntas visuales realizadas sobre imágenes tomadas por personas con discapacidades visuales durante la ejecución de sus tareas cotidianas. *(click [aqui](https://vizwiz.org/tasks-and-datasets/vqa/) para mas información sobre este dataset)*
    * Reseñas de Vinos (es):
      * ≈ 130k muestras.
      * Extraído de descripciones de vinos de todo el mundo.
* A continuación, seleccionen de la lista desplegable el dataset que mas nos interese.
* En `max_samples`, elijan un límite máximo de datos para acotar el cómputo necesario.
* Automáticamente el siguiente código dividirá el dataset elegido en dos partes:
    * El 80% como conjunto de entrenamiento.
    * El 20% como conjunto de test.


In [ ]:
# @markdown ❗
# @title  { run: "auto" }
import os
import torch
from datasets import load_dataset, Dataset

# Fijación de semilla para reproducibilidad de resultados
torch.manual_seed(42)

# Recolección de parámetros de formulario
DATASET_NAME = "Peppa Pig (es)" # @param ["Referencias de Vinos (es)", "Preguntas de no videntes (en)", "Martin Fierro (es)", "Peppa Pig (es)"]
max_samples = 30000 # @param {type:"number"}
name_to_file = {
    'Preguntas de no videntes (en)' : "vizwiz.csv",
    'Referencias de Vinos (es)'     : "wines_es.csv",
    'Martin Fierro (es)'            : "martin_fierro.csv",
    'Peppa Pig (es)'                : "peppa_pig.csv",
}

# Carga de conjunto de datos
bert_ds = load_dataset(
    path=DATASETS_PATH,
    data_files={'all_data': name_to_file[DATASET_NAME]},
)

# División de conjunto de datos en subconjuntos de entrenamiento y testeto
total_size = min(max_samples, len(bert_ds['all_data']))
val_size = int(total_size *.2)
train_size = total_size - val_size

bert_ds = bert_ds["all_data"].train_test_split(
    train_size=train_size,
    test_size=val_size,
    seed=42
)

print(f"* Información de dataset '{DATASET_NAME}':\n---")
bert_ds

In [ ]:
# @markdown 🔎
# Vizualización de primeros 15 ejemplos del dataset
for sample in bert_ds['train']['samples'][:15]:
    print(f">> {sample}")

### Tokenización de datos


- La tokenización es un paso de pre-procesamiento esencial cuando se utiliza BERT u otro LLM.
- BERT procesa el texto en forma de tokens, que son elementos individuales del texto de entrada, como palabras, signos de puntuación o tokens especiales como **[CLS]**, **[SEP]** , **[PAD]** , **[UNK]** , etc.
- Recuerde utilizar el modelo `bert-base-uncased` como modelo base para los datasets en idioma inglés, y `dccuchile/bert-base-spanish-wwm-uncased` para los datasets en español.

In [ ]:
# @markdown ❗
# Iniciación de tokenizador
from transformers import BertTokenizer

MODEL_CHECKPOINT = "dccuchile/bert-base-spanish-wwm-uncased" # @param ["bert-base-uncased","dccuchile/bert-base-spanish-wwm-uncased"]
tokenizer = BertTokenizer.from_pretrained(MODEL_CHECKPOINT, TOKENIZERS_PARALLELISM=False)

In [ ]:
# @markdown 🔎
# Tokenización de ejemplo de prueba
samples = bert_ds['train']['samples'][:5]

for ith, sample in enumerate(samples):
    token_ids = tokenizer(sample)['input_ids']
    token_str = [tokenizer.decode([tk_id]) for tk_id in token_ids]
    print(f"{sample}")
    print(f"{token_str}\n")

In [ ]:
# @markdown 🔎
# Visualización de 'tokes especiales' utilizados por tokenizador
for name, tk in tokenizer.special_tokens_map.items():
    print(f"{name}. Token: {tk}; Token_id: {tokenizer.vocab[tk]}")

* Dado que entrenaremos un modelo de BERT con un conjunto de datos que tienen una longitud variable, es fundamental aplicar el proceso de tokenización y el relleno (padding) adecuados a nuestros datos. Esto permite que todas las secuencias tengan la misma longitud fija, garantizando un procesamiento uniforme y eficiente de los datos durante el entrenamiento y la inferencia.
* A continuación realizaremos un histograma de la cantidad de tokens de cada ejemplo, y utilizaremos esa información para realizar posteriormente la tokenización del dataset completo.

In [ ]:
# @markdown ❗
# Generación de histograma de número de tokens por ejemplos
import matplotlib.pyplot as plt
import numpy as np

# Cálculo de número de tokens por ejemplos
def count_tokens_fn(batch):
    tokenized_batch = tokenizer(batch['samples'])
    tokenized_batch['count'] = [len(tks) for tks in tokenized_batch['input_ids']]
    return tokenized_batch

result = bert_ds.map(
    function=count_tokens_fn,
    batched=True
)

num_of_tokens_list = result['train']['count'] + result['test']['count']

# Generación de histograma
plt.hist(num_of_tokens_list, bins=35, edgecolor='k')
plt.xlabel('Número de Tokens')
plt.ylabel('Frecuencia')
plt.title('Número de Tokens por Ejemplo')
plt.show()

# Cálculo de percentiles (e.g., 25th, 50th, and 75th percentiles)
percentiles = np.percentile(num_of_tokens_list, [25, 50, 75])

print(f"Total de ejemplos: {len(num_of_tokens_list)}")
print(f"Percentil 25: {percentiles[0]}")
print(f"Percentil 50 (Mediana): {percentiles[1]}")
print(f"Percentile 75: {percentiles[2]}")
print(f"Max: {np.max(num_of_tokens_list)}")

- La función auxiliar `tokenize_data_fn` servirá para tokenizar y codificar el conjunto completo de datos de manera eficiente mediante el [método `map`](https://huggingface.co/docs/datasets/about_map_batch).
- Lo que devuelve es un nuevo dataset cuyos tokens estarán convertidos en:
    - índices del vocabulario (*input_ids*),
    - mascaras de atención [(*attention_mask*)](https://huggingface.co/docs/transformers/glossary#attention-mask),
    - tipos de tokens [(*token_type_ids*)](https://huggingface.co/docs/transformers/glossary#token-type-ids),
    - y etiquetas (*labels*), los cuales serán untilizados como `Ground True` en la etapa de entrenamiento.
- `MAX_TOKEN_LENGTH`: Indica el máximo número de tokens a utilizar por data. Debemos aumentarlo cuando nuestros conjuntos de datos son más largos. Una estrategia común es realizar un histograma y quedarnos con el valor del percentil 75% del largo de nuestros datos. También podemos utilizar el número de tokens de la seccuencia mas larga si se dispone de capacidad de cómputo suficiente.

In [ ]:
# @markdown ❗
# Tokenicación de conjunto de datos completo

# Número máximo de tokens a utilizar
MAX_TOKEN_LENGTH = 33 # @param {type:"number"}

def tokenize_data_fn(batch):
    tokenized_sample = tokenizer(
        batch['samples'],
        max_length=MAX_TOKEN_LENGTH,
        truncation=True,
        padding='max_length',
        return_tensors='pt',
    )

    tokenized_sample["labels"] = tokenized_sample['input_ids'].clone()
    return tokenized_sample

tokenized_bert_ds = bert_ds.map(
    function=tokenize_data_fn,
    batched=True,
    remove_columns=["samples"]
)

print(tokenized_bert_ds)

In [ ]:
# @markdown 🔎
# Visualización de proceso de tokenización para el primer ejemplo del conjunto de datos
sample = tokenized_bert_ds["train"][0]

print(f"- input_ids:        {sample['input_ids']}")
print(f"- attention_mask:   {sample['attention_mask']}")
print(f"- token_type_ids:   {sample['token_type_ids']}")
print(f"- labels:           {sample['labels']}")

### Enmascarado Aleatorio

> Este método se encarga de realizar el enmascaramiento aleatorio y formatear los datos de entrenamiento de manera adecuada, asegurando que las secuencias de entrada tengan la longitud correcta y que las máscaras de atención se apliquen correctamente.

* Para implementar este proceso de manera sencilla, utilizaremos el método [DataCollatorForLanguageModeling](https://huggingface.co/docs/transformers/v4.32.1/en/main_classes/data_collator#transformers.DataCollatorForLanguageModeling) proporcionado por la biblioteca HuggingFace.

* Como se podrá observar en el siguente ejemplo, es común utilizar un valor especial como **-100** en la salida de las etiquetas (*labels*) para indicar la posición de los tokens que se **IGNORAN** en el cálculo de la función de pérdida llevada a cabo durante el entrenamiento.

In [ ]:
# @markdown ❗
import torch, random
from transformers import DataCollatorForLanguageModeling

data_collate_fn = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=0.15
)

In [ ]:
# @markdown 🔎
# Visualizacion de proceso de 'enmascarado aleatorio' de tokens
seed = random.randint(0,100)
samples = [tokenized_bert_ds['train'][i] for i in range(3)]

print("DataCollator Outputs")
torch.manual_seed(seed)
print("--- Tokens:")
for sample in data_collate_fn(samples)['input_ids']:
    print(f">> {[tokenizer.decode([tk]) for tk in sample]}'")

torch.manual_seed(seed)
print("\n--- 'input_ids':")
for sample in data_collate_fn(samples)['input_ids']:
    print(f">> {sample.tolist()}'")

torch.manual_seed(seed)
print("\n--- 'labels':")
for sample in data_collate_fn(samples)['labels']:
    print(f">> {sample.tolist()}")

### Probando el Modelo Base

- Antes de ajustar el modelo a nuestro conjunto de datos seleccionados, veremos cómo se desenvuelve en la tarea de *fill-mask* **antes de la personalización**.

- Para ello, escriba una frase muy característica del dominio al que se quiere adaptar, y enmascare alguna de sus palabras utilizando el token `[MASK]`.


In [ ]:
# @markdown 🔎
# Ejemplo para español, pensando en adaptar el modelo a Peppa Pig
# Usamos pipeline como función auxiliar de alto nivel
from transformers import pipeline

fill_mask = pipeline(
    task="fill-mask",
    model="dccuchile/bert-base-spanish-wwm-uncased",
    top_k=5
)

ejemplo = 'A Peppa le encanta saltar en los [MASK] de barro'
fill_mask(ejemplo)

In [ ]:
# @markdown 🔎
# Ejemplo para inglés, pensando en adaptar el modelo a VizWiz
# Usamos pipeline como función auxiliar de alto nivel
from transformers import pipeline

fill_mask = pipeline(
    task="fill-mask",
    model="bert-base-uncased",
    top_k=5
)

ejemplo = "Can you [MASK] me the model on the back of the iPhone?"
fill_mask(ejemplo)

### Entrenamiento para adaptación

- Una vez definido el conjunto de datos, pasamos a la parte más intensa computacionalmente, el entrenamiento.
- Podemos decidir guardar el modelo localmente o hacer un backup de cada época del modelo en Hugging Face.
- Definimos las propiedades del entrenamiento mediante [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments).
- Definimos el entrenamiento del modelo mediante [`Trainer`](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer).
    
> <u>Detalles de parámetros a configurar</u>:
1.   ***epochs***: Cantidad de epochs (una epoch es una pasada completa del dataset para el algoritmo de entrenamiento)
2.  ***batch_size***: Cantidad de datos procesados por iteración antes de actualizar el modelo (aumentar este número mejora las estimaciones de cada iteración ya que será una muestra más representativa del dataset en general, pero requerirá más memoria)
3.  ***learning_rate***:  Tamaño del 'paso' en la actualización del modelo en cada iteración mientras avanza hacia un mínimo de la función de pérdida.

**Nota**: El valor de estos parámetros determinará el tiempo y la cantidad de cálculo requerido para poder realizar el entrenamiento del modelo. El mismo, puede ir desde unos segundos hasta varios minutos, dependiendo de los parámetros configurados y el poder de cómputo disponible.


In [ ]:
# @markdown ❗
from transformers import TrainingArguments, Trainer, BertForMaskedLM

# Eliminación de checkpoints anteriores
!rm -r "$CUSTOM_MODEL_CHECKPOINT"

# Configuración de hiperparámetros
MODEL_CHECKPOINT = "dccuchile/bert-base-spanish-wwm-uncased" # @param ["dccuchile/bert-base-spanish-wwm-uncased", "bert-base-uncased"]
BATCH_SIZE = 64 # @param {type:"slider", min:8, max:64, step:8}
LEARNING_RATE = 2e-5 # @param {type:"number"}
EPOCHS = 5 # @param {type:"slider", min:3, max:15, step:1}
LOGGING_STEPS = len(tokenized_bert_ds["train"]) // BATCH_SIZE
CUSTOM_MODEL_CHECKPOINT = "bert_adaptation_" + DATASET_NAME.replace(" ", "_")[:-5].lower()

# Configuración de entrenamiento
training_args = TrainingArguments(
    output_dir=CUSTOM_MODEL_CHECKPOINT,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    optim='adamw_torch',
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    push_to_hub=False,
    fp16=True,
    logging_steps=LOGGING_STEPS,
    report_to="none",
    save_strategy="no"
)

# Instanciación de modelo
model = BertForMaskedLM.from_pretrained(MODEL_CHECKPOINT)

# Instanciación de clase Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bert_ds["train"],
    eval_dataset=tokenized_bert_ds["test"],
    data_collator=data_collate_fn,
    tokenizer=tokenizer,
)

trainer.args._n_gpu = 1

* Antes de comenzar el entrenamiento calcularemos la [**Perplexity**](https://huggingface.co/docs/transformers/perplexity) del modelo de lenguaje.

> La `Perplexity` es una métrica que nos permite evaluar cuánta incertidumbre tiene un modelo cuando tiene que predecir palabras en un conjunto de prueba determinado. La misma, ayuda a medir qué tan bien conoce el modelo el lenguaje, y cuán coherentes serán sus predicciones. En general, una perplejidad más baja indicará que el modelo genera textos que claramente pertenecen al lenguaje.
<center>
<img src="https://thegradient.pub/content/images/size/w1600/2020/04/xkcd_entropy-2.png" width=50%>
</center>
fuente: https://thegradient.pub/understanding-evaluation-metrics-for-language-models/

Si queremos evaluar qué tanto se adapta un modelo a un dominio determinado, podemos ver la perplejidad que tiene el modelo para las tareas de completar palabras (MLM) o generación de texto en oraciones del dominio de interés, ya sea un conjunto de oraciones que nos interesen particularmente (un *benchmark* o *testbed*) o un corpus representativo del dominio en general.

* Luego del entrenamiento, estimaremos la `perplexity` del modelo y compararemos con los valores obtenidos previos al mismo.

In [ ]:
# @markdown  🔎
import math

# Fijamos semilla para reproducibilidad de resultados
torch.manual_seed(42)

# Estimación de perplexity previa a entrenamieto
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
# @markdown ❗⏳
# Inicio de entrenamiento
trainer.train()

# Guardado de entrenamiento
trainer.save_model(CUSTOM_MODEL_CHECKPOINT)

In [ ]:
# @markdown 🔎
# Fijamos semilla para reproducibilidad de resultados
torch.manual_seed(42)

# Estimación de perplexity posterior a entrenamiento
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


### Probando el Modelo Adaptado

- Ahora que tenemos el modelo entrenado con el dataset específico, la pregunta es, ¿Cómo se comportará?
- Para ello volvemos a hacer la prueba anterior utilizando la tarea de *fill-mask*, quizás esta vez con mejores resultados.
- Para que los resultados sean mas fáciles de comparar, la función `compare_models()` será la encargada de procesar y retornar, para cada frase de ejemplo ingresada, un ranking comparativo de las predicciones realizadas entre el nuevo modelo adaptado y el modelo base original.

In [ ]:
# @markdown ❗
import os
import transformers
from typing import List
from tabulate import tabulate
from transformers import pipeline

# Función auxiliar para comparación de predicciones de modelos
def compare_models(
    custom_model: transformers.pipelines.fill_mask.FillMaskPipeline,
    base_model: transformers.pipelines.fill_mask.FillMaskPipeline,
    sentences: List[str],
    top_k=5
) -> None:

    headers = ['Sent','Custom Bert', '(%)', 'Base Bert', '(%)']
    rows = []
    for sent in sentences:
        rows.append([sent, '', '', '', ''])

        c_out = custom_model(sent)
        o_out = base_model(sent)

        c_tokens = [(i['token_str'], str(round(i['score']*100,1))+"%") for i in c_out]
        o_tokens = [(i['token_str'], str(round(i['score']*100,1))+"%") for i in o_out]

        for (p1,s1), (p2,s2) in zip(c_tokens, o_tokens):
            rows.append(['', p1, s1, p2, s2])

    table = tabulate(rows, headers=headers, tablefmt="text", numalign="center")
    print(table, "\n")

# Instanciación de modelo personalizado y modelo base
base_model = pipeline(task="fill-mask", model=MODEL_CHECKPOINT)
custom_model = pipeline(task="fill-mask", model=CUSTOM_MODEL_CHECKPOINT)

In [ ]:
# @markdown ❗

# Conjunto de ejemplos de prueba
vizwiz_test_examples = [
    "can you tell me the title of the book? [MASK].",
    "please [MASK] this shirt",
    "what [MASK] is this?",
    "[MASK] can you tell me what is this?",
    "how clear is this [MASK]?",
    "is the floor [MASK]?",
    "the microwave is [MASK]?."
]

vinos_test_examples = [
    "Este [MASK] argentino de altura es una verdadera",
    "Con un color profundo e [MASK]",
    "Los sabores de [MASK] persisten",
    "Hecho 100% de [MASK]",
    "El sabor fue [MASK]",
    "Con su [MASK] mezcla de minerales",
    "Por primera vez en [MASK],",
    "Brazamora de [MASK] bonito"
]

martin_fierro_test_examples = [
    "En los campos de la [MASK] extensa",
    "donde el viento y el sol se [MASK]",
    "cabalgo firme con mi [MASK]",
    "bajo el cielo de [MASK] risueño",

    "Con mi [MASK] y mi lazo",
    "enfrento a la [MASK] con valor",
    "batiéndome siempre a mi [MASK]",
    "en este ancho y rudo [MASK]",

    "Así cabalgo con alma [MASK]",
    "como otros en su [MASK]",
    "defendiendo mi [MASK] y mi gente",
    "en esta tierra que [MASK] sin cesar",

    "Con [MASK] al cinto, firme y diestro",
    "Enfrento [MASK], firme como el resto",
    "Mate en mano, bajo el cielo [MASK]",
    "Mi coraje y mi [MASK], jamás han claudicado."
]

peppa_pig_test_examples = [
    "[MASK], puedes decir dinosarurio?",
    "¡Hola, soy Peppa [MASK]!",
    "Hoy es un día [MASK] para un picnic.",
    "George tiene un [MASK] muy querido.",
    "El sol brilla en el [MASK] del jardín.",
    "Peppa y George juegan en el [MASK].",
    "Mamá Pig prepara [MASK] deliciosos.",
    "Vamos al [MASK] para montar en bicicleta.",
    "El [MASK] está lleno de flores hermosas.",
    "Peppa y sus amigos tienen [MASK] en el parque."
]

# Moficar 'sentences' acorde al conjunto de datos utilizado para la personalización
compare_models(
    custom_model=custom_model,
    base_model=base_model,
    sentences=peppa_pig_test_examples
)

### Subi tu modelo a Hugging Face  🤗

> Recorda generar tu **token de acceso** siguendo las instrucciones de la [Sección 2](#Librerías-para-modelos-de-lenguaje). Será necesario para que puedas autentificarte en la plataforma y subir tu modelo.

In [ ]:
# @markdown ❗
# Logeo a plataforma mediante token de acceso
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# @markdown ❗
# Esto envía el modelo entrenado al repositorio de modelos de Hugging Face.
trainer.push_to_hub()

## 4.2 - ¿Cómo personalizar un modelo GPT?


- Se inicia por algún modelo pre-entrenado para la tarea específica que uno busca (e.g. clasificación, generación, etc).
- Se toma un corpus especializado (anotado, revisado, etc.) y se entrena utilizando dicho corpus.
- Intentaremos [entrenar que un modelo genere texto](https://github.com/huggingface/notebooks/blob/main/examples/language_modeling.ipynb) con el estilo de alguno de los conjuntos de datos disponibles para este taller.
- Utilizaremos el modelo `DeepESP/gpt2-spanish` como base para español y el modelo `gpt2` como base para inglés.
* A diferencia de la adaptación de dominio realizada en el modelo BERT, los modelos de generación de texto se entrenan utilizando la tarea auto-supervisada llamada [**Causal Language Modeling**](https://huggingface.co/docs/transformers/tasks/language_modeling).


In [3]:
# @markdown ❗
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

BASE_MODEL = "DeepESP/gpt2-spanish" # @param ["DeepESP/gpt2-spanish", "gpt2"]
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)

### Probando el Modelo Base

- Antes de ajustar el modelo vemos cómo se desenvuelve si le damos como entrada el primer dato del dataset elegido.

In [ ]:
# @markdown 🔎

# Fijamos semilla para reproducibilidad de resultados
torch.manual_seed(42)

input_ids = tokenizer.encode("Ayer, al depertarme", return_tensors='pt')
sampling_output = model.generate(input_ids, do_sample=True, max_length=50, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
output = tokenizer.decode(sampling_output[0], skip_special_tokens=True)

print(output)

* Utilizaremos la librería `datasets` de HuggingFace para cargar cada uno de los corpus seleccionados.
* Para la personalización de GPT, tenemos un total de 6 conjuntos de datos, todos en español:
    * Julio Cortázar (es):
      * ≈ 14k oraciones.
      * Extraído de una selección de libros y cuentos del autor argentino.
    * Edgar Allan Poe (es):
      * ≈ 48k oraciones.
      * Extraído de una selección de libros y cuentos del autor estadounidense traducidos al español.
    * José Saramago (es):
      * ≈ 105k oraciones.
      * Extraído de una selección de libros y cuentos del autor portugués traducidos al español.
    * Peppa Pig (es):
      * ≈ 3k oraciones.
      * Extraído de subtítulos de 77 episodios.
    * Martin Fierro (es):
      * ≈ 2k oraciones.
      * Extraído del libro completo de José Hernández
    * Reseñas de Vinos (es):
      * ≈ 130k oraciones.
      * Extraído de descripciones de vinos de todo el mundo.

* A continuación, seleccionen de la lista desplegable el dataset que mas nos interese.
* En `max_samples`, elijan un límite máximo de datos para acotar el cómputo necesario.
* Automáticamente el siguiente código dividirá el dataset elegido en dos partes:
    * El 80% como conjunto de entrenamiento.
    * El 20% como conjunto de test.


In [ ]:
# @markdown ❗
# @title  { run: "auto" }
import os
import torch
from datasets import load_dataset, Dataset

# Fijamos semilla para reproducibilidad de resultados
torch.manual_seed(42)

# Recolección de parámetros de formulario
dataset_name = "Jose Saramago (es)" # @param ["Reseñas de Vinos (es)", "VizWiz (en)", "Martin Fierro (es)", "Peppa Pig (es)", "Julio Cortazar (es)", "Edgar Allan Poe (es)", "Jose Saramago (es)"]
max_samples = 30000 # @param {type:"number"}
name_to_file = {
    'Referencias de Vinos (es)' : "wines_es.csv",
    'Martin Fierro (es)'        : "martin_fierro.csv",
    'Peppa Pig (es)'            : "peppa_pig.csv",
    'Julio Cortazar (es)'       : "cortazar.csv",
    'Edgar Allan Poe (es)'      : "poe.csv",
    'Jose Saramago (es)'        : "saramago.csv",
}

# Carga de conjunto de datos
gpt2_ds = load_dataset(
    path=DATASETS_PATH,
    data_files={'all_data': name_to_file[dataset_name]},
)

# División de conjunto de datos en subconjuntos de entrenamiento y testeto
total_size = min(max_samples, len(gpt2_ds['all_data']))
val_size = int(total_size *.2)
train_size = total_size - val_size

gpt2_ds = gpt2_ds["all_data"].train_test_split(
    train_size=train_size,
    test_size=val_size,
    seed=42
)

print(f"* Información del dataset '{dataset_name}':\n---")
gpt2_ds

In [ ]:
# @markdown 🔎
# Visualización de primeros 15 ejemplos del dataset
for sample in gpt2_ds['train']['samples'][:15]:
    print(f">> {sample}")

### Tokenizando los datos


- Al igual que en la primera parte de este taller, tokenizaremos el dataset. Para más detalles de este proceso, ir a la sección `Tokenización de datos` en la primera mitad de este notebook.
- Realizaremos un histograma de la cantidad de tokens de cada ejemplo, y utilizaremos esa información para realizar posteriormente la tokenización del dataset completo.


In [ ]:
# @markdown ❗
# Generación de histograma del número de tokens por ejemplos
import matplotlib.pyplot as plt
import numpy as np

# Cálculo de número de tokens por ejemplos
def count_tokens_fn(batch):
    tokenized_batch = tokenizer(batch['samples'])
    tokenized_batch['count'] = [len(tks) for tks in tokenized_batch['input_ids']]
    return tokenized_batch

result = gpt2_ds.map(
    function=count_tokens_fn,
    batched=True
)

num_of_tokens_list = result['train']['count'] + result['test']['count']

# Generación de histograma
plt.hist(num_of_tokens_list, bins=35, edgecolor='k')
plt.xlabel('Número de Tokens')
plt.ylabel('Frecuencia')
plt.title('Número de Tokens por Ejemplo')
plt.show()

# Cálculo de percentiles (e.g., 25th, 50th, and 75th percentiles)
percentiles = np.percentile(num_of_tokens_list, [25, 50, 75])

print(f"Total de ejemplos: {len(num_of_tokens_list)}")
print(f"Percentil 25: {percentiles[0]}")
print(f"Percentil 50 (Mediana): {percentiles[1]}")
print(f"Percentile 75: {percentiles[2]}")
print(f"Max: {np.max(num_of_tokens_list)}")

- En `MAX_TOKEN_LENGTH` podrán elegir el largo máximo de cada dato. Truncando todos aquellos más largos, y realizando padding para aquellos más cortos.

In [ ]:
# @markdown ❗
# Tokenicación de conjunto de datos completo

# Número máximo de tokens a utilizar
MAX_TOKEN_LENGTH = 30 # @param {type:"slider", min:8, max:128, step:8}

def tokenize_data_fn(batch):
    tokenized_sample = tokenizer(
        batch['samples'],
        max_length=MAX_TOKEN_LENGTH,
        truncation=True,
        padding='max_length',
        return_tensors='pt',
    )

    tokenized_sample["labels"] = tokenized_sample['input_ids'].clone()
    return tokenized_sample

tokenized_gpt2_ds = gpt2_ds.map(
    function=tokenize_data_fn,
    batched=True,
    remove_columns=["samples"]
)

print(tokenized_gpt2_ds)

### Decodificando

- Podemos ver que los textos pasan a estar agrupados en bloques de `MAX_TOKEN_LENGTH` tokens.
- Además, vemos que el texto fue reemplazado por números (índices en el vocabulario).
- Por último, si decodificamos estos números, obtenemos el texto original.

In [ ]:
# @markdown 🔎
print(len(tokenized_gpt2_ds['train'][0]['input_ids']))
print(tokenized_gpt2_ds['train'][0]['input_ids'][:10])

30
[5438, 318, 268, 8800, 774, 388, 2241, 23, 11301, 970]


In [ ]:
# @markdown 🔎
print(tokenizer.decode(tokenized_gpt2_ds['train'][0]['input_ids']))

gesto de raíz volteriana. Años antes, en Ensayo sobre la lucidez (2004), se había adentrado en la inconsistencia y las des


### Entrenamiento

- Una vez definido el conjunto de datos, pasamos a la parte más intensa computacionalmente, el entrenamiento.
- Podemos decidir guardar el modelo localmente o hacer un backup de cada época del modelo en Hugging Face.
- Definimos las propiedades del entrenamiento mediante [`TrainingArguments`](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments).
- Definimos el entrenamiento del modelo mediante [`Trainer`](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.Trainer).
    - El entrenamiento tardará desde unos segundos hasta varias horas dependiendo el poder de cómputo, parámetros elegidos y tamaño del dataset.

In [ ]:
# @markdown ❗⏳
from transformers import Trainer, TrainingArguments

LEARNING_RATE = 2e-5 # @param {type:"number"}
EPOCHS = 5 # @param {type:"slider", min:3, max:15, step:1}

model_name = dataset_name.replace(" ", "-")[:-5]

training_args = TrainingArguments(
    model_name,
    evaluation_strategy='epoch',
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    logging_steps=5
)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_gpt2_ds['train'],
    eval_dataset=tokenized_gpt2_ds['test']
)

trainer.args._n_gpu = 1

trainer.train()

- Ahora será necesario ingresar a la plataforma HuggingFace 🤗 para subir nuestro modelo. Luego podrán utilizarlo tanto en este taller con la librería `transformers` como compartir con sus amigos en la versión web.

In [ ]:
# @markdown ❗

# Logeo a plataforma mediante token de acceso
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# @markdown ❗

# Esto envía el modelo entrenado al repositorio de modelos de Hugging Face.
trainer.push_to_hub()
tokenizer.push_to_hub(model_name)

### Probando el Nuevo Modelo

- Ahora que tenemos el modelo entrenado, la pregunta es, ¿Cómo se comportará?
- Para ello volvemos a hacer la prueba anterior, quizás esta vez con mejores resultados.
- En `MODEL` deberán pegar el link al modelo que acaban de subir. Tendrá un formato del estilo `username/modelName`.

In [ ]:
# @markdown ❗
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL = "guidoivetta/Peppa-Pig" # @param {type:"string"}
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForCausalLM.from_pretrained(MODEL)

- En esta celda podrán probar su modelo. Les incentivamos a ser creativos con las frases que elijan para completarla y comparar los resultados del modelo base con el modelo adaptado.

In [ ]:
# @markdown ❗

# Fijamos semilla para reproducibilidad de resultados
torch.manual_seed(42)  # To ensure determinism

input_ids = tokenizer.encode("Ayer al despertarme", return_tensors='pt')
sampling_output = model.generate(input_ids, do_sample=True, max_length=50, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
output = tokenizer.decode(sampling_output[0], skip_special_tokens=True)

print(output)

> Este trabajo está licenciado bajo la [Licencia MIT](https://github.com/nanom/llm_adaptation_workshop/blob/main/LICENSE).
